In [0]:
use catalog my_catalog;
use schema default;

In [0]:
select current_catalog()

create a customer table

In [0]:
CREATE OR REPLACE TABLE customer_data (
  customer_id INT,
  name STRING,
  email STRING,
  city STRING,
  last_updated TIMESTAMP
) TBLPROPERTIES (
  delta.enableChangeDataFeed = true
);

INSERT INTO customer_data VALUES
  (1, 'Alice', 'alice@gmail.com', 'New York', current_timestamp()),
  (2, 'Bob', 'bob@gmail.com', 'Chicago', current_timestamp()),
  (3, 'Carol', 'carol@gmail.com', 'Austin', current_timestamp());


create a customer mirror downstream table

In [0]:
CREATE OR REPLACE TABLE customer_data_mirror AS
SELECT * FROM customer_data;


In [0]:
select * from customer_data;

In [0]:
select * from customer_data_mirror;

Perform dml operations on source table

In [0]:
-- Update Bob's city
UPDATE customer_data
SET city = 'San Francisco', last_updated = current_timestamp()
WHERE customer_id = 2;

-- Insert a new customer
INSERT INTO customer_data VALUES
  (4, 'David', 'david@gmail.com', 'Seattle', current_timestamp());

-- Delete Carol
DELETE FROM customer_data WHERE customer_id = 3;


In [0]:
desc history customer_data

In [0]:
select * from table_changes('customer_data',2)

In [0]:
select * from customer_data

In [0]:
select * from customer_data_mirror

merge changes into mirror table

In [0]:
merge into customer_data_mirror as target
using(
select * from table_changes('customer_data',2)
where _change_type in ('insert','update_postimage')
) as source on source.customer_id = target.customer_id
when matched then update set *
when not matched then insert *;

delete delete records


In [0]:
DELETE FROM customer_data_mirror
WHERE customer_id IN (
  SELECT customer_id FROM table_changes('customer_data', 2)
  WHERE _change_type = 'delete'
);


In [0]:
select * from customer_data

In [0]:
select * from customer_data_mirror

In [0]:
select * from table_changes('customer_data',1)
where _change_type in ('insert','update_postimage')
qualify row_number() over(partition by customer_id order by _commit_timestamp desc) = 1